In [5]:
import pytesseract
from PIL import Image
import os

In [6]:
# Path to the Tesseract executable (change this according to your setup)
# pytesseract.pytesseract.tesseract_cmd = r'C:\Users\USER\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'
#ocr_tesseract_path = os.path.join(os.getcwd(), "tesseract_ocr/tesseract.exe")
#pytesseract.pytesseract.tesseract_cmd = ocr_tesseract_path
pytesseract.pytesseract.tesseract_cmd = '/usr/local/bin/tesseract'
#def parse_receipt(image_path):
 #   with Image.open(image_path) as img:
  #      text = pytesseract.image_to_string(img)
   #     return text

# Path to your receipt image
#image_path = os.path.join(os.getcwd(), "sample_receipts/walmart01.png")
#receipt_text = parse_receipt(image_path)
#print(receipt_text)

from PIL import Image
import pytesseract

def parse_receipt(image_path):
    with Image.open(image_path) as img:
        text = pytesseract.image_to_string(img)
        return text

# Path to your receipt image
image_path = os.path.join(os.getcwd(), "sample_receipts/walmart01.png")
receipt_text = parse_receipt(image_path)
print(receipt_text)


Walmart >.<"

Save money. Live better.
(330 ) 339 - 3991
MANAGER DIANA EARNEST
231 BLUEBELL DR SW
NEW PHILADELPHIA OH 44663
ST# 02115 OP# 009044 THY 44 TRH 01301

PET TOY 004747571658 1.97 X
FLOPPY PUPPY 004747514846 1.97 X
SSSUPREME S 070060332153 4.97 x
2.5 SQUEAK 084699803238 5.92 X
MUNCHY DMBEL 068113108796 3.77
DOG TREAT 007119013654 292 X
PED PCH 1 002310011802 0.50 x
PED PCH 1 002310011802 0.50 x
COUPON 23100 052310037000 1.00-0
HNYMD SHORES 088491226837 F alow o
FRENCH DRSNG 004132100655 F 1.98 0
3 ORANGES 001466835001 F 5.47 N
BABY CARROTS 003338366602 I 1.48.8
COLLARDS 000000004614KT LizaN
CALZONE 005208362080 F 2.50 0
MM RVW MNT 003399105848 19.77 x
STKOBRLPLABL 001558679414 1197 X
STKOBRLPLABL 001558679414 1.97 x
STKO SUNFLWR 001558679410 0197 x
STKO SUNFLWR 001558679410 0.97 x
STKO SUNFLWR 001558679410 0197 x
STKO SUNFLWR 001558679410 0.97 X
BLING BEADS 076594060699 0197 x
GREAT VALUE 007874203191 F 9.97 0
LIPTON 001200011224 F 4l4u x
DRY Do 002310011035 12.44.

SUBTOTAL 9

In [41]:
import spacy
from spacy.matcher import Matcher

# Load the English NLP model
nlp = spacy.load("en_core_web_sm")

# The text to be processed (example receipt text)
text = receipt_text

# Create the spaCy document
doc = nlp(text)

#Finding the last index of a lemma of 'total'
last_index = len(doc) - 1
for token in doc:
    if token.text.lower() == 'total' or token.lemma_ == 'total':
        last_index = token.i 
        
# Splitting the text upto the last total into lines
lines = doc[0:last_index].text.strip().split('\n')

# Initialize the Matcher
matcher = Matcher(nlp.vocab)
item_pattern = [{'TEXT': {'REGEX': r'^\d{1,4}(\.\d+)?$'}}]
matcher.add("POSSIBLE_PRICES", [item_pattern])

#Finding the item name and price
for line in lines:
    doc2 = nlp(line)
    matches = matcher(doc2)
    for match_id, start, end in matches:
        span = doc2[start:end]  
        alphabetic_tokens = [token.text for token in doc2[0:start] if token.is_alpha]
        if len(alphabetic_tokens) > 0:
            print(f"Item: {alphabetic_tokens}")
            print(f"Price:  {span.text}")


Item: ['ST', 'OP', 'THY']
Price:  44
Item: ['PET', 'TOY']
Price:  1.97
Item: ['FLOPPY', 'PUPPY']
Price:  1.97
Item: ['SSSUPREME', 'S']
Price:  4.97
Item: ['SQUEAK']
Price:  5.92
Item: ['MUNCHY', 'DMBEL']
Price:  3.77
Item: ['DOG', 'TREAT']
Price:  292
Item: ['PED', 'PCH']
Price:  1
Item: ['PED', 'PCH']
Price:  0.50
Item: ['PED', 'PCH']
Price:  1
Item: ['PED', 'PCH']
Price:  0.50
Item: ['COUPON']
Price:  1.00
Item: ['COUPON']
Price:  0
Item: ['FRENCH', 'DRSNG', 'F']
Price:  1.98
Item: ['FRENCH', 'DRSNG', 'F']
Price:  0
Item: ['ORANGES', 'F']
Price:  5.47
Item: ['CALZONE', 'F']
Price:  2.50
Item: ['CALZONE', 'F']
Price:  0
Item: ['MM', 'RVW', 'MNT']
Price:  19.77
Item: ['STKOBRLPLABL']
Price:  1197
Item: ['STKOBRLPLABL']
Price:  1.97
Item: ['STKO', 'SUNFLWR']
Price:  0197
Item: ['STKO', 'SUNFLWR']
Price:  0.97
Item: ['STKO', 'SUNFLWR']
Price:  0197
Item: ['STKO', 'SUNFLWR']
Price:  0.97
Item: ['BLING', 'BEADS']
Price:  0197
Item: ['GREAT', 'VALUE', 'F']
Price:  9.97
Item: ['GREAT', 'VALU